<a href="https://colab.research.google.com/github/sarthak0806/KDSH/blob/main/Bored_From_Ludo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Dropout, Activation
from prophet import Prophet
from sklearn.ensemble import RandomForestRegressor

# Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/btc_1h.csv')
df3=dataset.copy()

In [ ]:
data_list = []
for name in dataset.columns:
  if name not in (['datetime','volume']):
    name = dataset[name]
    data_list.append(name)

In [ ]:
feat_names = ['open','high','low','close']

# EDA, Feature Engineering and Data Preparation

In [ ]:
def seasonal_decompose_data(data,type = 'additive',period = 100):
  decomposed = seasonal_decompose(data, model = type, period = period)
  fig = plt.figure()
  fig = decomposed.plot()
  fig.set_size_inches(15,10)

In [ ]:
for data in data_list:
  seasonal_decompose_data(data)

In [ ]:
def plot_auto_cor(data):
  pd.plotting.autocorrelation_plot(data)

In [ ]:
plot_auto_cor(data_list[0])

In [ ]:
def Scaling(data):
  sc = MinMaxScaler(feature_range = (0, 1))
  scaled_data = sc.fit_transform(data)
  return scaled_data, sc


In [ ]:
def train_test_split(dataset,percent_size=0.8):
  dataset=pd.DataFrame(dataset)
  n_train_rows = int(0.8*len(dataset))

  train = dataset[:n_train_rows]
  test = dataset[n_train_rows:]

  return train, test

In [ ]:
def xy(data,steps=24):
  x_train = []
  y_train = []

  for i in range(steps, data.shape[0]-steps):
      x_train.append(data.iloc[i-steps:i, :])
      y_train.append(data.iloc[i, :])

  x_train, y_train = np.array(x_train), np.array(y_train)
  return x_train,y_train

In [ ]:
def RMSE(test,pred):
  rmse = np.sqrt(np.mean(((pred - test) ** 2)))
  return rmse

# LSTM

In [ ]:
dataset=df3
dataset=dataset.drop(columns=['datetime','open','high','volume','low'])
dataset,sc=Scaling(dataset)

train, test = train_test_split(dataset,0.8)

x_train,y_train=xy(train)
x_test,y_test=xy(test)

In [ ]:
model = Sequential()
model.add(LSTM(units=128,activation='relu', return_sequences = True, input_shape = (x_train.shape[1],1)))
model.add(LSTM(units=64, return_sequences = False))
model.add(Dense(units=25))
model.add(Dense(units=1))
model.compile(loss="mean_squared_error", optimizer="adam")
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 24, 128)           66560     
                                                                 
 lstm_13 (LSTM)              (None, 64)                49408     
                                                                 
 dense_12 (Dense)            (None, 25)                1625      
                                                                 
 dense_13 (Dense)            (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
nepochs = 10
model.fit(x_train, y_train, epochs = nepochs)

In [ ]:
y_hat = model.predict(x_test)
y_hat = sc.inverse_transform(y_hat)
y_test=sc.inverse_transform(y_test)

222/222 [==============================] - 7s 26ms/step


In [ ]:
print("RMSE from LSTM: ")
print((RMSE(y_test,y_hat)))

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(y_test, color = 'red', label = 'Actual')
plt.plot(y_hat, color = 'blue', label = 'Predictions')
plt.title('LSTM')
plt.ylabel('close')
plt.legend()
plt.show()

# Random Forest

In [ ]:
dataset=df3
dataset=dataset.drop(columns=['datetime','open','high','volume','low'])
dataset,sc=Scaling(dataset)

train, test = train_test_split(dataset,0.8)

x_train,y_train=xy(train)
x_test,y_test=xy(test)

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1])
y_train = y_train.reshape(y_train.shape[0])

x_test, y_test = np.array(x_test), np.array(y_test)
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1])
y_test = y_test.reshape(y_test.shape[0])

In [ ]:
model2 = RandomForestRegressor(n_estimators = 250, random_state=0)

In [ ]:
model2.fit(x_train,y_train)
result=model2.predict(x_test)

result=sc.inverse_transform([result])
y_test=sc.inverse_transform([y_test])

result = result.T
y_test = y_test.T

In [ ]:
print("RMSE from Random Forest: ")
print(RMSE(result,y_test))

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(y_test, color = 'red', label = 'Actual')
plt.plot(result, color = 'blue', label = 'Predictions')
plt.title('Random Forest')
plt.ylabel('close')
plt.legend()
plt.show()

# FB Prophet


In [ ]:
def FBP_scaler(data):
  scaler = MinMaxScaler(feature_range = (0,1))
  data.loc[:,'y']  = scaler.fit_transform(pd.DataFrame(data.loc[:,'y']))
  return data, scaler

In [ ]:
def feat_create(data, nlags = 24):
  X = pd.DataFrame([])

  for i in range(0, nlags + 1):
    X = pd.concat([X, data['y'].rename(str(i)).shift(-i)],axis = 1)

  X = pd.concat([data.shift(-(nlags+1)),X], axis = 1)
  X = X[:-(nlags + 1)]

  train_len = int(len(X) * 0.8)
  X_train = X[:train_len]
  X_test = X[train_len:]

  valid = df3[['datetime','close']].shift(-(train_len + nlags+1))
  valid = valid.iloc[:-(nlags + 1 + train_len),:]
  X_test = X_test.drop(['y'], axis = 1)

  return X_train, X_test, valid

In [ ]:
data = df3[['datetime','close']]
data.columns = ['ds', 'y']
data,scaler = FBP_scaler(data)

X_train, X_test, valid = feat_create(data)

<ipython-input-114-6ed7cdf02481>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:,'y']  = scaler.fit_transform(pd.DataFrame(data.loc[:,'y']))


In [ ]:
model = Prophet()
for feature in X_train.columns:
    if feature not in ["ds","y"]:
      model.add_regressor(feature)
model.fit(X_train)

DEBUG:cmdstanpy:input tempfile: /tmp/tmpsd74fdhb/9lf1jx2f.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsd74fdhb/ka_a5_6d.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=87699', 'data', 'file=/tmp/tmpsd74fdhb/9lf1jx2f.json', 'init=/tmp/tmpsd74fdhb/ka_a5_6d.json', 'output', 'file=/tmp/tmpsd74fdhb/prophet_modelnor63hmc/prophet_model-20240112205941.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
20:59:41 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
21:01:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [ ]:
forecast = model.predict(X_test)
prediction = scaler.inverse_transform(forecast[['yhat']])

valid['pred'] = prediction
valid = valid.set_index('datetime')

print("RMSE from FB Prophet Model : ")
print(RMSE(valid['close'],valid['pred']))

In [ ]:
plt.figure(figsize=(18,9))
plt.title('FB Prophet')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(valid['close'], color = 'red', label = 'Actual')
plt.plot(valid['pred'], color = 'blue', label = 'Predictions')
plt.legend([ 'Val', 'Predictions'], loc='lower right')
plt.show()


# FInal

In [ ]:
finalpred=(y_hat + prediction[:y_hat.shape[0]] + result)/3

In [ ]:
print("Final RMSE: ")
print(RMSE(finalpred,y_test))

In [ ]:
plt.figure(figsize = (18,9))
plt.plot(y_test, color = 'red', label = 'Actual')
plt.plot(finalpred, color = 'blue', label = 'Predictions')
plt.title('Final Model')
plt.ylabel('close')
plt.legend()
plt.show()

# Strategy and Backtesting


In [ ]:
pip install backtesting

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
from backtesting import Backtest,Strategy
from backtesting.lib import crossover
import talib
from talib import EMA, RSI, MACD

In [ ]:
data = df3
data['datetime']=pd.to_datetime(data['datetime'])
data.index=data.pop('datetime')
data.columns = ['Open','High','Low','Close','Volume']
data.index.name = None

In [ ]:
class new(Strategy):
  sema_close_period = 75
  fema_close_period = 5
  rsi_period = 20
  min_rsi = 23
  max_rsi = 72
  macd_fast = 17
  macd_slow = 25
  stoploss = 3
  take_profit = 20
  def init(self):

    close = self.data.Close
    high = self.data.High
    low = self.data.Low
    self.slowma = self.I(EMA, close, self.sema_close_period)
    self.fastma = self.I(EMA, close, self.fema_close_period)
    self.macd = self.I(MACD, close,self.macd_slow)
    self.macdsig = self.I(MACD,close,self.macd_fast)
    self.rsi_val = self.I(RSI, close, self.rsi_period)

  def next(self):
      curr_price = self.data.Close[-1]

      if crossover(self.fastma, self.slowma) :
        self.position.close()
        self.buy(sl = (100-self.stoploss)*curr_price/100, tp = ((100+self.take_profit)*curr_price/100))
      elif crossover(self.slowma, self.fastma) :
        self.position.close()
        self.sell(sl = (100+self.stoploss)*curr_price/100, tp = ((100-self.take_profit)*curr_price/100))

      if self.rsi_val > self.max_rsi and self.macd > self.macdsig:
        self.position.close()
        self.sell(sl = (100+self.stoploss)*curr_price/100, tp = ((100-self.take_profit)*curr_price/100))

      if self.rsi_val < self.min_rsi and self.macd < self.macdsig:
        self.position.close()
        self.buy(sl = (100-self.stoploss)*curr_price/100, tp = ((100+self.take_profit)*curr_price/100))

In [ ]:
backtest = Backtest(data,new,commission = 0.00,cash=100000,exclusive_orders=True)
stats = backtest.run()
print(stats)

Start                     2018-01-01 05:30:00
End                       2022-01-31 05:30:00
Duration                   1491 days 00:00:00
Exposure Time [%]                   87.589726
Equity Final [$]                22866949.7429
Equity Peak [$]                 26035297.0879
Return [%]                       22766.949743
Buy & Hold Return [%]              172.224649
Return (Ann.) [%]                  277.704491
Volatility (Ann.) [%]              254.285028
Sharpe Ratio                         1.092099
Sortino Ratio                        8.368836
Calmar Ratio                         7.818789
Max. Drawdown [%]                  -35.517582
Avg. Drawdown [%]                   -3.239976
Max. Drawdown Duration      176 days 01:00:00
Avg. Drawdown Duration        3 days 11:00:00
# Trades                                  906
Win Rate [%]                        27.041943
Best Trade [%]                      20.093566
Worst Trade [%]                     -3.162216
Avg. Trade [%]                    